Try to directly compare the cosine sim between the user's query and the ad's glove and BERT embeddings

1.Glove embedding

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ECE1786\ Project/datasets/ad\ dataset1

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ECE1786 Project/datasets/ad dataset1


In [ ]:
!pip install gensim transformers torch
import gensim.downloader as api
import numpy as np
glove_model = api.load("glove-wiki-gigaword-300")

[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
def get_glove_embedding(sentence):
    words = sentence.lower().split()
    word_vectors = [glove_model[word] for word in words if word in glove_model]
    if len(word_vectors) == 0:
        return np.zeros(300)
    sentence_embedding = np.mean(word_vectors, axis=0)
    return sentence_embedding

2.Bert embedding

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased").to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
def get_bert_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to('cuda')
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.squeeze(0)
    sentence_embedding = embeddings.mean(dim=0)
    return sentence_embedding.cpu().numpy()

In [ ]:
import pandas as pd
file_path = 'train_250k_query_emb.csv'
commercial_ads_data = pd.read_csv(file_path, sep=',', on_bad_lines='skip', header = None)
commercial_ads_data.columns = commercial_ads_data.iloc[0]
commercial_ads_data = commercial_ads_data[1:].reset_index(drop=True)
commercial_ads_data = commercial_ads_data.dropna()
commercial_ads_data.head()


,NaN,NaN,Search_Query,Ads_Title,Ads_Description,website,NaN,company,label,NaN,search_query_bert_embedding,query_glove_embedding,query_combined_embedding
0,2935782.0,1942925.0,Portable Wrench Holder,Klutch 15-Slot Universal Wrench Pouch,Sturdy Klutch fabric pouch is ideal for organi...,northerntool.com,Northern Tool,Klutch,1,[IMG_CATEID_3746],[-4.51255813e-02 1.63031295e-01 -2.25683451e-...,[ 5.08419871e-02 1.01236671e-01 9.87449884e-...,[ 0.05084199 0.10123667 0.09874499 ... 0.11...
1,4509227.0,4665926.0,fuel gauges,Harley-Davidson Skull LED Fuel Gauge,This low-profile fuel gauge replaces the analo...,harley-davidson.com,Harley-Davidson,HARLEY-DAVIDSON,1,[IMG_CATEID_3323],[ 2.53215998e-01 -1.54950336e-01 -2.29238272e-...,[ 0.5353 -0.0146405 0.66892004 -0.697275...,[ 0.5353 -0.0146405 0.66892004 ... -0.15...
2,2060197.0,249418.0,engagement rings,Entwined Halo Diamond Engagement Ring - 14K Ro...,Entwined Halo Diamond Engagement Ring - 14K Ro...,brilliantearth.com,Brilliant Earth,Brilliant Earth,1,[IMG_CATEID_372],[ 4.97438937e-01 9.32224691e-02 -6.47221878e-...,[-0.680345 -0.07579 0.1446395 -0.593055...,[-0.680345 -0.07579 0.1446395 ... -0.07...
3,416413.0,177680.0,carpet liquidators,Emser Tile T06FONT0404UT Trav Fontane Tumbled ...,"Emser Tile, T06FONT0404UT, Tile, Trav Fontane ...",build.com,Build,Emser Tile,1,[IMG_CATEID_729],[ 2.40466788e-01 -1.64019033e-01 -2.30942890e-...,[ 0.34874648 0.237075 0.32426 -0.123515...,[ 0.34874648 0.237075 0.32426 ... 0.33...
4,2094971.0,1737068.0,moen shower faucet repair,MOEN Kingsley 2-Handle Deck-Mount Roman Tub Tr...,The Kingsley bath series from MOEN offers a ne...,homedepot.com,Home Depot,MOEN,1,[IMG_CATEID_383],[ 4.53501850e-01 -4.88466956e-02 -1.53228357e-...,[ 0.13240375 -0.09338725 -0.17774926 -0.379427...,[ 0.13240375 -0.09338725 -0.17774926 ... 0.14...


In [ ]:
commercial_ads_data = commercial_ads_data[commercial_ads_data['label'] == '1']
commercial_ads_data.head()

,NaN,NaN,Search_Query,Ads_Title,Ads_Description,website,NaN,company,label,NaN,search_query_bert_embedding,query_glove_embedding,query_combined_embedding
0,2935782.0,1942925.0,Portable Wrench Holder,Klutch 15-Slot Universal Wrench Pouch,Sturdy Klutch fabric pouch is ideal for organi...,northerntool.com,Northern Tool,Klutch,1,[IMG_CATEID_3746],[-4.51255813e-02 1.63031295e-01 -2.25683451e-...,[ 5.08419871e-02 1.01236671e-01 9.87449884e-...,[ 0.05084199 0.10123667 0.09874499 ... 0.11...
1,4509227.0,4665926.0,fuel gauges,Harley-Davidson Skull LED Fuel Gauge,This low-profile fuel gauge replaces the analo...,harley-davidson.com,Harley-Davidson,HARLEY-DAVIDSON,1,[IMG_CATEID_3323],[ 2.53215998e-01 -1.54950336e-01 -2.29238272e-...,[ 0.5353 -0.0146405 0.66892004 -0.697275...,[ 0.5353 -0.0146405 0.66892004 ... -0.15...
2,2060197.0,249418.0,engagement rings,Entwined Halo Diamond Engagement Ring - 14K Ro...,Entwined Halo Diamond Engagement Ring - 14K Ro...,brilliantearth.com,Brilliant Earth,Brilliant Earth,1,[IMG_CATEID_372],[ 4.97438937e-01 9.32224691e-02 -6.47221878e-...,[-0.680345 -0.07579 0.1446395 -0.593055...,[-0.680345 -0.07579 0.1446395 ... -0.07...
3,416413.0,177680.0,carpet liquidators,Emser Tile T06FONT0404UT Trav Fontane Tumbled ...,"Emser Tile, T06FONT0404UT, Tile, Trav Fontane ...",build.com,Build,Emser Tile,1,[IMG_CATEID_729],[ 2.40466788e-01 -1.64019033e-01 -2.30942890e-...,[ 0.34874648 0.237075 0.32426 -0.123515...,[ 0.34874648 0.237075 0.32426 ... 0.33...
4,2094971.0,1737068.0,moen shower faucet repair,MOEN Kingsley 2-Handle Deck-Mount Roman Tub Tr...,The Kingsley bath series from MOEN offers a ne...,homedepot.com,Home Depot,MOEN,1,[IMG_CATEID_383],[ 4.53501850e-01 -4.88466956e-02 -1.53228357e-...,[ 0.13240375 -0.09338725 -0.17774926 -0.379427...,[ 0.13240375 -0.09338725 -0.17774926 ... 0.14...


In [ ]:
#concat both embeddings
glove_weight = 0.7
bert_weight = 0.3
commercial_ads_data['search_query_bert_embedding'] = commercial_ads_data['search_query_bert_embedding'].apply(
    lambda x: np.fromstring(x.strip("[]"), sep=" ") if isinstance(x, str) else x
)
commercial_ads_data['query_glove_embedding'] = commercial_ads_data['query_glove_embedding'].apply(
    lambda x: np.fromstring(x.strip("[]"), sep=" ") if isinstance(x, str) else x
)
commercial_ads_data['query_combined_embedding'] = commercial_ads_data.apply(
    lambda row: np.concatenate([
        row['query_glove_embedding'] * glove_weight,
        row['search_query_bert_embedding'] * bert_weight
    ]),
    axis=1
)

'''
commercial_ads_data['search_query_bert_embedding'] = commercial_ads_data['Search_Query'].apply(get_bert_embedding)
commercial_ads_data['query_glove_embedding'] = commercial_ads_data['Search_Query'].apply(get_glove_embedding)
commercial_ads_data['query_combined_embedding'] = commercial_ads_data.apply(
    lambda row: np.concatenate([row['query_glove_embedding'], row['search_query_bert_embedding']]),
    axis=1)
output_file_path = 'train_250k_query_emb.csv'
commercial_ads_data.to_csv(output_file_path, index=False)
'''


"\ncommercial_ads_data['search_query_bert_embedding'] = commercial_ads_data['Search_Query'].apply(get_bert_embedding)\ncommercial_ads_data['query_glove_embedding'] = commercial_ads_data['Search_Query'].apply(get_glove_embedding)\ncommercial_ads_data['query_combined_embedding'] = commercial_ads_data.apply(\n    lambda row: np.concatenate([row['query_glove_embedding'], row['search_query_bert_embedding']]),\n    axis=1)\noutput_file_path = 'train_250k_query_emb.csv'\ncommercial_ads_data.to_csv(output_file_path, index=False)\n"

In [ ]:
commercial_ads_data.head()

,NaN,NaN,Search_Query,Ads_Title,Ads_Description,website,NaN,company,label,NaN,search_query_bert_embedding,query_glove_embedding,query_combined_embedding
0,2935782.0,1942925.0,Portable Wrench Holder,Klutch 15-Slot Universal Wrench Pouch,Sturdy Klutch fabric pouch is ideal for organi...,northerntool.com,Northern Tool,Klutch,1,[IMG_CATEID_3746],"[-0.0451255813, 0.163031295, -0.225683451, 0.2...","[0.0508419871, 0.101236671, 0.0987449884, -0.1...","[0.035589390969999996, 0.0708656697, 0.0691214..."
1,4509227.0,4665926.0,fuel gauges,Harley-Davidson Skull LED Fuel Gauge,This low-profile fuel gauge replaces the analo...,harley-davidson.com,Harley-Davidson,HARLEY-DAVIDSON,1,[IMG_CATEID_3323],"[0.253215998, -0.154950336, -0.229238272, 0.17...","[0.5353, -0.0146405, 0.66892004, -0.697275, -0...","[0.37471, -0.01024835, 0.468244028, -0.4880924..."
2,2060197.0,249418.0,engagement rings,Entwined Halo Diamond Engagement Ring - 14K Ro...,Entwined Halo Diamond Engagement Ring - 14K Ro...,brilliantearth.com,Brilliant Earth,Brilliant Earth,1,[IMG_CATEID_372],"[0.497438937, 0.0932224691, -0.0647221878, 0.0...","[-0.680345, -0.07579, 0.1446395, -0.593055, -0...","[-0.47624149999999993, -0.053052999999999996, ..."
3,416413.0,177680.0,carpet liquidators,Emser Tile T06FONT0404UT Trav Fontane Tumbled ...,"Emser Tile, T06FONT0404UT, Tile, Trav Fontane ...",build.com,Build,Emser Tile,1,[IMG_CATEID_729],"[0.240466788, -0.164019033, -0.23094289, 0.107...","[0.34874648, 0.237075, 0.32426, -0.12351501, -...","[0.244122536, 0.1659525, 0.226982, -0.08646050..."
4,2094971.0,1737068.0,moen shower faucet repair,MOEN Kingsley 2-Handle Deck-Mount Roman Tub Tr...,The Kingsley bath series from MOEN offers a ne...,homedepot.com,Home Depot,MOEN,1,[IMG_CATEID_383],"[0.45350185, -0.0488466956, -0.153228357, -0.0...","[0.13240375, -0.09338725, -0.17774926, -0.3794...","[0.09268262499999999, -0.065371075, -0.1244244..."


In [ ]:
query = 'remote student job'

query_embedding = get_glove_embedding(query) * glove_weight
query_embedding_bert = get_bert_embedding(query) * bert_weight
query_combined_embedding = np.concatenate([query_embedding, query_embedding_bert])




#try the concat of them
combined_embeddings = np.vstack(commercial_ads_data['query_combined_embedding'].values)
cos_sim_combined = np.dot(combined_embeddings, query_combined_embedding) / (
    np.linalg.norm(combined_embeddings, axis=1) * np.linalg.norm(query_combined_embedding)
)
most_similar_combined_idx = np.argmax(cos_sim_combined)
most_similar_combined_value = cos_sim_combined[most_similar_combined_idx]

most_similar_combined_description = commercial_ads_data.loc[most_similar_combined_idx, 'Ads_Description']
most_similar_combined_query = commercial_ads_data.loc[most_similar_combined_idx, 'Search_Query']
print(f"{most_similar_combined_description} \nMost similar query in database: {most_similar_combined_query} \n(similarity: {most_similar_combined_value})"
)



The Dell Latitude E6440 is a business class laptop which offers excellent processing power, a large and comfortable keyboard, durable, and reliable. It's the perfect notebook for web browsing, streaming, multitasking and more. The Dell E6440 i5 originally sold new for over $1700. Today you can buy one from us at a fraction of that price with our one year warranty included.Specifications:Intel Core i5-4300M 2.6Ghz CPU4GB Memory250gb SATA Hard Drive14 Inch HD Display1366 x 768 ResolutionHDMI & VGA Video Output3x USB 3.0 PortsMemory Card ReaderRJ-45 Ethernet Connector802.11g/n Integrated WiFiThe charger/power cord is included.Windows 10 Home Pre-loadedAC Adapter Charger included 
Most similar query in database: computer sales near me 
(similarity: 0.6756409695650167)


Shirley

In [ ]:
import pandas as pd
import numpy as np

user_queries_df = pd.read_csv("user_query_shirley.csv")


def extract_queries(generated_queries):
    return generated_queries.split("; ")

for index, row in user_queries_df.iterrows():
    user_name = row['Name']
    queries = extract_queries(row['Generated_Queries'])

    print(f"Processing queries for {user_name}...\n")

    for query in queries:
        print(f"Query: {query}")

        query_embedding = get_glove_embedding(query)
        query_embedding_bert = get_bert_embedding(query)
        query_combined_embedding = np.concatenate([query_embedding, query_embedding_bert])

        combined_embeddings = np.vstack(commercial_ads_data['query_combined_embedding'].values)

        cos_sim_combined = np.dot(combined_embeddings, query_combined_embedding) / (
            np.linalg.norm(combined_embeddings, axis=1) * np.linalg.norm(query_combined_embedding)
        )

        most_similar_combined_idx = np.argmax(cos_sim_combined)
        most_similar_combined_value = cos_sim_combined[most_similar_combined_idx]

        most_similar_combined_description = commercial_ads_data.loc[most_similar_combined_idx, 'Ads_Description']
        most_similar_combined_query = commercial_ads_data.loc[most_similar_combined_idx, 'Search_Query']

        print(f"Most similar ad with combined embedding: {most_similar_combined_description}")
        print(f"Most similar query in database: {most_similar_combined_query} (similarity: {most_similar_combined_value:.4f})\n")


FileNotFoundError: [Errno 2] No such file or directory: 'user_query_shirley.csv'